In [1]:
import sys

# sys.path.append("/scratch/group/csce435-f24/python-3.10.4/lib/python3.10/site-packages")
sys.path.append("C:/Users/BJMin/AppData/Local/Programs/Python/Python312/Lib/site-packages")
# sys.path.append("/scratch/group/csce435-f24/thicket")
from glob import glob

import matplotlib.pyplot as plt 
import pandas as pd

import thicket as th

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [3]:
tk = th.Thicket.from_caliperreader(glob("cali_files/bitonic_cali/*.cali"))

AttributeError: module 'thicket' has no attribute 'Thicket'

In [2]:
print(tk.tree(metric_column="Avg time/rank"))

NameError: name 'tk' is not defined

In [ ]:
tk.metadata_column_to_perfdata("algorithm")
tk.metadata_column_to_perfdata("input_size")
tk.metadata_column_to_perfdata("input_type")
tk.metadata_column_to_perfdata("num_procs")

tk.dataframe = tk.dataframe.reset_index().set_index(["node", "algorithm", "input_size", "input_type", "num_procs"]).sort_index()

tk.dataframe.head()

In [ ]:
# Define common variables
processes = [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]
array_sizes = [65536, 262144, 1048576, 4194304, 16777216, 67108864, 268435456]
input_types = ["sorted", "random", "reverse", "perturbed"]

In [ ]:
# Change font size for all plots
plt.rcParams.update({"font.size": 20})

In [ ]:
def plot_pandas(df, title, xlabel, ylabel):
    plot = df.plot(kind="line", 
        marker='o', 
        title=title, 
        xlabel=xlabel,
        ylabel=ylabel, 
        figsize=(15,7),
        grid=True
    )
    return plot

In [ ]:
for node in tk.graph.traverse():
    if node.frame["name"] != 'main' and node.frame["name"] != 'comm' and node.frame["name"] != 'comp_large':
        continue
    print(node)

In [ ]:
# graphs for main. comm, and comp_large
for node in tk.graph.traverse():
    if node.frame["name"] != 'main' and node.frame["name"] != 'comm' and node.frame["name"] != 'comp_large':
        continue
    for size in array_sizes:
        df=(tk.dataframe.loc[node, "Avg time/rank"])
        df = (df.loc[df.index.get_level_values('input_size') == size])
        df = df.unstack(level="input_type")
        df = df.unstack(level="input_size")
        df = df.unstack(level="algorithm")
        plot_pandas(
            df,
            title="Sample Sort - " + str(node.frame["name"])+" - input size "+str(size),
            xlabel="Number of Processes",
            ylabel="Time (seconds)",
        )

        # automatically save graph as a file
        filename = f"./graphs/Sample_{str(node.frame['name'])}_{str(size)}.png"
        plt.savefig(filename)  # Save the plot as a PNG file